In [1]:
import numpy as np
import pandas as pd

pd.set_option("display.max.columns", 100)
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split

In [2]:
df = pd.read_csv("../../data/bank.csv")

In [66]:
df.head(-5)

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,0
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4511,46,blue-collar,married,secondary,no,668,yes,no,unknown,15,may,1263,2,-1,0,unknown,1
4512,40,blue-collar,married,secondary,no,1100,yes,no,unknown,29,may,660,2,-1,0,unknown,0
4513,49,blue-collar,married,secondary,no,322,no,no,cellular,14,aug,356,2,-1,0,unknown,0
4514,38,blue-collar,married,secondary,no,1205,yes,no,cellular,20,apr,45,4,153,1,failure,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   int64 
dtypes: int64(8), object(9)
memory usage: 600.6+ KB


Всего 9 признаков со строковыми значениями.

In [5]:
df.columns[df.dtypes == "object"]

Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact',
       'month', 'poutcome'],
      dtype='object')

## Без категориальных признаков
Попытаемся сначала просто проигнорировать категориальные признаки. Обучим случайный лес и посмотрим на ROC AUC на кросс-валидации и на отоженной выборке. Это будет наш бейзлайн. 

In [6]:
df_no_cat, y = df.loc[:, df.dtypes != "object"].drop("y", axis=1), df["y"]

In [7]:
df_no_cat_part, df_no_cat_valid, y_train_part, y_valid = train_test_split(
    df_no_cat, y, test_size=0.3, stratify=y, random_state=17
)

In [8]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=17)

In [9]:
forest = RandomForestClassifier(random_state=17)

In [10]:
np.mean(
    cross_val_score(forest, df_no_cat_part, y_train_part, cv=skf, scoring="roc_auc")
)

0.8495016786335677

In [11]:
forest.fit(df_no_cat_part, y_train_part)

RandomForestClassifier(random_state=17)

In [12]:
roc_auc_score(y_valid, forest.predict_proba(df_no_cat_valid)[:, 1])

0.8631508998911164

## LabelEncoder для категориальных признаков
Сделаем то же самое, но попробуем закодировать категориальные признаки по-простому: с помощью `LabelEncoder`.

In [13]:
from sklearn.preprocessing import LabelEncoder

In [14]:
label_encoder = LabelEncoder()

In [15]:
df_cat_label_enc = df.copy().drop("y", axis=1)
for col in df.columns[df.dtypes == "object"]:
    df_cat_label_enc[col] = label_encoder.fit_transform(df_cat_label_enc[col])

In [16]:
df_cat_label_enc.shape

(4521, 16)

In [17]:
df_cat_label_enc_part, df_cat_label_enc_valid = train_test_split(
    df_cat_label_enc, test_size=0.3, stratify=y, random_state=17
)

In [18]:
np.mean(
    cross_val_score(
        forest, df_cat_label_enc_part, y_train_part, cv=skf, scoring="roc_auc"
    )
)

0.8922975749958866

In [19]:
forest.fit(df_cat_label_enc_part, y_train_part)

RandomForestClassifier(random_state=17)

In [20]:
roc_auc_score(y_valid, forest.predict_proba(df_cat_label_enc_valid)[:, 1])

0.908855334230022

## Бинаризация категориальных признаков (dummies, OHE)
Теперь сделаем то, что обычно по умолчанию и делают – бинаризацию категориальных признаков. Dummy-признаки, One-Hot Encoding... с небольшими различиями это об одном же - для каждого значения каждого категориального признака завести свой бинарный признак.

In [21]:
df_cat_dummies = pd.get_dummies(df, columns=df.columns[df.dtypes == "object"]).drop(
    "y", axis=1
)

In [22]:
df_cat_dummies.shape

(4521, 51)

In [23]:
df_cat_dummies_part, df_cat_dummies_valid = train_test_split(
    df_cat_dummies, test_size=0.3, stratify=y, random_state=17
)

In [24]:
np.mean(
    cross_val_score(
        forest, df_cat_dummies_part, y_train_part, cv=skf, scoring="roc_auc"
    )
)

0.8988421716862304

In [25]:
forest.fit(df_cat_dummies_part, y_train_part)

RandomForestClassifier(random_state=17)

In [26]:
roc_auc_score(y_valid, forest.predict_proba(df_cat_dummies_valid)[:, 1])

0.9172511155233887

## Попарные взаимодействия признаков
Пока лес все еще лучше регрессии (хотя мы не тюнили гиперпараметры, но и не будем). Мы хотим идти дальше. Мощной техникой для работы с категориальными признаками будет учет попарных взаимодействий признаков (feature interactions). Построим попарные взаимодействия всех признаков. Вообще тут можно пойти дальше и строить взаимодействия трех и более признаков. Owen Zhang [как-то строил](https://www.youtube.com/watch?v=LgLcfZjNF44) даже 7-way interactions. Чего не сделаешь ради победы на Kaggle! :)

In [27]:
df_interact = df.copy()

In [28]:
cat_features = df.columns[df.dtypes == "object"]
for i, col1 in enumerate(cat_features):
    for j, col2 in enumerate(cat_features[i + 1 :]):
        df_interact[col1 + "_" + col2] = df_interact[col1] + "_" + df_interact[col2]

In [29]:
df_interact.shape

(4521, 53)

In [30]:
df_interact.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,job_marital,job_education,job_default,job_housing,job_loan,job_contact,job_month,job_poutcome,marital_education,marital_default,marital_housing,marital_loan,marital_contact,marital_month,marital_poutcome,education_default,education_housing,education_loan,education_contact,education_month,education_poutcome,default_housing,default_loan,default_contact,default_month,default_poutcome,housing_loan,housing_contact,housing_month,housing_poutcome,loan_contact,loan_month,loan_poutcome,contact_month,contact_poutcome,month_poutcome
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,0,unemployed_married,unemployed_primary,unemployed_no,unemployed_no,unemployed_no,unemployed_cellular,unemployed_oct,unemployed_unknown,married_primary,married_no,married_no,married_no,married_cellular,married_oct,married_unknown,primary_no,primary_no,primary_no,primary_cellular,primary_oct,primary_unknown,no_no,no_no,no_cellular,no_oct,no_unknown,no_no,no_cellular,no_oct,no_unknown,no_cellular,no_oct,no_unknown,cellular_oct,cellular_unknown,oct_unknown
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,0,services_married,services_secondary,services_no,services_yes,services_yes,services_cellular,services_may,services_failure,married_secondary,married_no,married_yes,married_yes,married_cellular,married_may,married_failure,secondary_no,secondary_yes,secondary_yes,secondary_cellular,secondary_may,secondary_failure,no_yes,no_yes,no_cellular,no_may,no_failure,yes_yes,yes_cellular,yes_may,yes_failure,yes_cellular,yes_may,yes_failure,cellular_may,cellular_failure,may_failure
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,0,management_single,management_tertiary,management_no,management_yes,management_no,management_cellular,management_apr,management_failure,single_tertiary,single_no,single_yes,single_no,single_cellular,single_apr,single_failure,tertiary_no,tertiary_yes,tertiary_no,tertiary_cellular,tertiary_apr,tertiary_failure,no_yes,no_no,no_cellular,no_apr,no_failure,yes_no,yes_cellular,yes_apr,yes_failure,no_cellular,no_apr,no_failure,cellular_apr,cellular_failure,apr_failure
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,0,management_married,management_tertiary,management_no,management_yes,management_yes,management_unknown,management_jun,management_unknown,married_tertiary,married_no,married_yes,married_yes,married_unknown,married_jun,married_unknown,tertiary_no,tertiary_yes,tertiary_yes,tertiary_unknown,tertiary_jun,tertiary_unknown,no_yes,no_yes,no_unknown,no_jun,no_unknown,yes_yes,yes_unknown,yes_jun,yes_unknown,yes_unknown,yes_jun,yes_unknown,unknown_jun,unknown_unknown,jun_unknown
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,0,blue-collar_married,blue-collar_secondary,blue-collar_no,blue-collar_yes,blue-collar_no,blue-collar_unknown,blue-collar_may,blue-collar_unknown,married_secondary,married_no,married_yes,married_no,married_unknown,married_may,married_unknown,secondary_no,secondary_yes,secondary_no,secondary_unknown,secondary_may,secondary_unknown,no_yes,no_no,no_unknown,no_may,no_unknown,yes_no,yes_unknown,yes_may,yes_unknown,no_unknown,no_may,no_unknown,unknown_may,unknown_unknown,may_unknown


## Бинаризация категориальных признаков (dummies, OHE) + попарные взаимодействия
Получилось аж 824 бинарных признака – многовато для такой задачи, и тут случайный лес начинает не справляться, да и логистическая регрессия сработала хуже, чем в прошлый раз.

In [31]:
df_interact_cat_dummies = pd.get_dummies(
    df_interact, columns=df_interact.columns[df_interact.dtypes == "object"]
).drop("y", axis=1)

In [32]:
df_interact_cat_dummies.shape

(4521, 824)

In [33]:
df_interact_cat_dummies_part, df_interact_cat_dummies_valid = train_test_split(
    df_interact_cat_dummies, test_size=0.3, stratify=y, random_state=17
)

In [34]:
np.mean(
    cross_val_score(
        forest, df_interact_cat_dummies_part, y_train_part, cv=skf, scoring="roc_auc"
    )
)

0.833999698931206

In [35]:
forest.fit(df_interact_cat_dummies_part, y_train_part)

RandomForestClassifier(random_state=17)

In [36]:
roc_auc_score(y_valid, forest.predict_proba(df_interact_cat_dummies_valid)[:, 1])

0.8618112043382651

Случайному лесу уже тяжеловато, когда признаков так много, а вот логистической регрессии – норм.

In [37]:
from sklearn.linear_model import LogisticRegression

logit = LogisticRegression(random_state=17)

In [38]:
np.mean(
    cross_val_score(
        logit, df_interact_cat_dummies_part, y_train_part, cv=skf, scoring="roc_auc"
    )
)

C:\Users\GVL\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\GVL\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.ht

0.860022081295576

In [39]:
logit.fit(df_interact_cat_dummies_part, y_train_part)

C:\Users\GVL\AppData\Roaming\Python\Python38\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=17)

In [40]:
roc_auc_score(y_valid, logit.predict_proba(df_interact_cat_dummies_valid)[:, 1])

0.8530284591899913

## Mean Target
Теперь будем использовать технику кодирования категориальных признаков средним значением целевого признака. Это очень мощная техника, правда, надо умело ее использовать – легко переобучиться. 
Основная идея – для каждого значения категориального признака посчитать среднее значение целевого признака и заменить категориальный признак на посчитанные средние. Правда, считать средние надо на кросс-валидации, а то легко переобучиться. 
Но далее я адресую к видео топ-участников соревнований Kaggle, от них можно узнать про эту технику из первых уст. 
- [Специализация](https://www.coursera.org/specializations/aml) "Advanced Machine Learning" на Coursera, [курс](https://www.coursera.org/learn/competitive-data-science)", How to Win a Data Science Competition: Learn from Top Kagglers", несколько видео посвящено различным способам построяния признаков с задействованием целевого, и как при этом не переобучиться. Рассказывает Дмитрий Алтухов
- [Лекция](https://www.youtube.com/watch?v=g335THJxkto) с презентацией решения конкурса Kaggle BNP paribas, Станислав Семенов

Похожая техника [используется](https://tech.yandex.com/catboost/doc/dg/concepts/algorithm-main-stages_cat-to-numberic-docpage/) и в CatBoost.

Для начала давайте таким образом закодируем исходные категориальные признаки.

In [41]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [42]:
train_df, y = df.copy(), df["y"]
train_df_part, valid_df, y_train_part, y_valid = train_test_split(
    train_df.drop("y", axis=1), y, test_size=0.3, stratify=y, random_state=17
)

In [43]:
def mean_target_enc(train_df, y_train, valid_df, skf):
    import warnings

    warnings.filterwarnings("ignore")

    glob_mean = y_train.mean()
    train_df = pd.concat([train_df, pd.Series(y_train, name="y")], axis=1)
    new_train_df = train_df.copy()

    cat_features = train_df.columns[train_df.dtypes == "object"].tolist()

    for col in cat_features:
        new_train_df[col + "_mean_target"] = [
            glob_mean for _ in range(new_train_df.shape[0])
        ]

    for train_idx, valid_idx in skf.split(train_df, y_train):
        train_df_cv, valid_df_cv = (
            train_df.iloc[train_idx, :],
            train_df.iloc[valid_idx, :],
        )

        for col in cat_features:

            means = valid_df_cv[col].map(train_df_cv.groupby(col)["y"].mean())
            valid_df_cv[col + "_mean_target"] = means.fillna(glob_mean)

        new_train_df.iloc[valid_idx] = valid_df_cv

    new_train_df.drop(cat_features + ["y"], axis=1, inplace=True)

    for col in cat_features:
        means = valid_df[col].map(train_df.groupby(col)["y"].mean())
        valid_df[col + "_mean_target"] = means.fillna(glob_mean)

    valid_df.drop(train_df.columns[train_df.dtypes == "object"], axis=1, inplace=True)

    return new_train_df, valid_df

In [44]:
train_mean_target_part, valid_mean_target = mean_target_enc(
    train_df_part, y_train_part, valid_df, skf
)

In [45]:
np.mean(
    cross_val_score(
        forest, train_mean_target_part, y_train_part, cv=skf, scoring="roc_auc"
    )
)

0.8813941498132323

In [46]:
forest.fit(train_mean_target_part, y_train_part)

RandomForestClassifier(random_state=17)

In [47]:
roc_auc_score(y_valid, forest.predict_proba(valid_mean_target)[:, 1])

0.9108221780994471

## Mean Target + попарные взаимодействия

In [48]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [49]:
train_df, y = df_interact.drop("y", axis=1).copy(), df_interact["y"]
train_df_part, valid_df, y_train_part, y_valid = train_test_split(
    train_df, y, test_size=0.3, stratify=y, random_state=17
)

In [50]:
train_mean_target_part, valid_mean_target = mean_target_enc(
    train_df_part, y_train_part, valid_df, skf
)

In [51]:
np.mean(
    cross_val_score(
        forest, train_mean_target_part, y_train_part, cv=skf, scoring="roc_auc"
    )
)

0.8864378441723935

In [52]:
forest.fit(train_mean_target_part, y_train_part)

RandomForestClassifier(random_state=17)

In [53]:
roc_auc_score(y_valid, forest.predict_proba(valid_mean_target)[:, 1])

0.9138538397489271

Опять лучше справляется логистическая регрессия.

In [54]:
np.mean(
    cross_val_score(
        logit, train_mean_target_part, y_train_part, cv=skf, scoring="roc_auc"
    )
)

0.7841777758717867

In [55]:
logit.fit(train_mean_target_part, y_train_part)

LogisticRegression(random_state=17)

In [56]:
roc_auc_score(y_valid, logit.predict_proba(valid_mean_target)[:, 1])

0.7494128824270373

## Catboost
В библиотеке [Catboost](https://catboost.yandex), помимо всего прочего, реализована как раз техника кодирования категориальных значений средним значением целевого признака. Результаты получаются хорошими именно когда в данных много важных категориальных признаков. Из минусов можно отметить меньшую (пока что) производительность в сравнении с Xgboost и LightGBM.

In [58]:
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/101.0 MB ? eta -:--:--
     -------------------------------------- 0.0/101.0 MB 960.0 kB/s eta 0:01:46
     ---------------------------------------- 0.2/101.0 MB 1.6 MB/s eta 0:01:02
     ---------------------------------------- 0.3/101.0 MB 1.9 MB/s eta 0:00:54
     ---------------------------------------- 0.3/101.0 MB 1.9 MB/s eta 0:00:54
     ---------------------------------------- 0.4/101.0 MB 1.7 MB/s eta 0:00:58
     ---------------------------------------- 0.6/101.0 MB 2.0 MB/s eta 0:00:50
     ---------------------------------------- 0.6/101.0 MB 2.0 MB/s eta 0:00:50
     ---------------------------------------- 0.7/101.0 MB 1.8 MB/s eta 0:00:58
     ---------------------------------------- 0.8/101.0 MB 1.9 MB/s eta 0:00:53
     ---------------------------------------- 0.8/101.0 MB 1.9 MB/s eta 0:00:53
     ---------------------------------------- 1.0

In [59]:
from catboost import CatBoostClassifier

In [60]:
ctb = CatBoostClassifier(random_seed=17)

In [61]:
train_df, y = df.drop("y", axis=1), df["y"]
train_df_part, valid_df, y_train_part, y_valid = train_test_split(
    train_df, y, test_size=0.3, stratify=y, random_state=17
)

In [62]:
cat_features_idx = np.where(train_df_part.dtypes == "object")[0].tolist()

In [63]:
%%time
cv_scores = []
for train_idx, test_idx in skf.split(train_df_part, y_train_part):
    cv_train_df, cv_valid_df = (
        train_df_part.iloc[train_idx, :],
        train_df_part.iloc[test_idx, :],
    )
    y_cv_train, y_cv_valid = y_train_part.iloc[train_idx], y_train_part.iloc[test_idx]

    ctb.fit(cv_train_df, y_cv_train, cat_features=cat_features_idx)

    cv_scores.append(roc_auc_score(y_cv_valid, ctb.predict_proba(cv_valid_df)[:, 1]))

Learning rate set to 0.015316
0:	learn: 0.6762525	total: 169ms	remaining: 2m 48s
1:	learn: 0.6618182	total: 191ms	remaining: 1m 35s
2:	learn: 0.6502370	total: 202ms	remaining: 1m 7s
3:	learn: 0.6362366	total: 220ms	remaining: 54.7s
4:	learn: 0.6199279	total: 250ms	remaining: 49.8s
5:	learn: 0.6093899	total: 260ms	remaining: 43.1s
6:	learn: 0.5942204	total: 283ms	remaining: 40.1s
7:	learn: 0.5814123	total: 309ms	remaining: 38.3s
8:	learn: 0.5681355	total: 334ms	remaining: 36.7s
9:	learn: 0.5548194	total: 359ms	remaining: 35.5s
10:	learn: 0.5440275	total: 385ms	remaining: 34.6s
11:	learn: 0.5327116	total: 410ms	remaining: 33.8s
12:	learn: 0.5225683	total: 443ms	remaining: 33.6s
13:	learn: 0.5110561	total: 473ms	remaining: 33.3s
14:	learn: 0.5047516	total: 487ms	remaining: 31.9s
15:	learn: 0.4966251	total: 504ms	remaining: 31s
16:	learn: 0.4897499	total: 526ms	remaining: 30.4s
17:	learn: 0.4801462	total: 552ms	remaining: 30.1s
18:	learn: 0.4707871	total: 577ms	remaining: 29.8s
19:	learn: 

163:	learn: 0.2253359	total: 4.84s	remaining: 24.7s
164:	learn: 0.2247049	total: 4.88s	remaining: 24.7s
165:	learn: 0.2243187	total: 4.91s	remaining: 24.7s
166:	learn: 0.2239158	total: 4.94s	remaining: 24.7s
167:	learn: 0.2234760	total: 4.97s	remaining: 24.6s
168:	learn: 0.2229897	total: 5.01s	remaining: 24.6s
169:	learn: 0.2225713	total: 5.04s	remaining: 24.6s
170:	learn: 0.2223428	total: 5.08s	remaining: 24.6s
171:	learn: 0.2221127	total: 5.11s	remaining: 24.6s
172:	learn: 0.2217149	total: 5.14s	remaining: 24.6s
173:	learn: 0.2215291	total: 5.18s	remaining: 24.6s
174:	learn: 0.2211475	total: 5.21s	remaining: 24.6s
175:	learn: 0.2206561	total: 5.24s	remaining: 24.5s
176:	learn: 0.2203367	total: 5.28s	remaining: 24.5s
177:	learn: 0.2202044	total: 5.3s	remaining: 24.5s
178:	learn: 0.2198978	total: 5.34s	remaining: 24.5s
179:	learn: 0.2198149	total: 5.36s	remaining: 24.4s
180:	learn: 0.2194927	total: 5.39s	remaining: 24.4s
181:	learn: 0.2193552	total: 5.44s	remaining: 24.5s
182:	learn: 0

325:	learn: 0.1932985	total: 10.1s	remaining: 20.8s
326:	learn: 0.1932949	total: 10.1s	remaining: 20.8s
327:	learn: 0.1932360	total: 10.1s	remaining: 20.7s
328:	learn: 0.1931401	total: 10.2s	remaining: 20.7s
329:	learn: 0.1930753	total: 10.2s	remaining: 20.7s
330:	learn: 0.1927619	total: 10.2s	remaining: 20.7s
331:	learn: 0.1926376	total: 10.3s	remaining: 20.7s
332:	learn: 0.1925292	total: 10.3s	remaining: 20.6s
333:	learn: 0.1924406	total: 10.3s	remaining: 20.6s
334:	learn: 0.1921516	total: 10.4s	remaining: 20.6s
335:	learn: 0.1919353	total: 10.4s	remaining: 20.6s
336:	learn: 0.1918214	total: 10.4s	remaining: 20.5s
337:	learn: 0.1917792	total: 10.5s	remaining: 20.5s
338:	learn: 0.1915678	total: 10.5s	remaining: 20.5s
339:	learn: 0.1915184	total: 10.5s	remaining: 20.4s
340:	learn: 0.1913235	total: 10.6s	remaining: 20.4s
341:	learn: 0.1912159	total: 10.6s	remaining: 20.4s
342:	learn: 0.1910944	total: 10.6s	remaining: 20.3s
343:	learn: 0.1908747	total: 10.7s	remaining: 20.3s
344:	learn: 

486:	learn: 0.1729909	total: 16s	remaining: 16.9s
487:	learn: 0.1728386	total: 16.1s	remaining: 16.9s
488:	learn: 0.1726625	total: 16.1s	remaining: 16.8s
489:	learn: 0.1725617	total: 16.2s	remaining: 16.8s
490:	learn: 0.1724188	total: 16.2s	remaining: 16.8s
491:	learn: 0.1722780	total: 16.3s	remaining: 16.8s
492:	learn: 0.1722164	total: 16.3s	remaining: 16.8s
493:	learn: 0.1720885	total: 16.3s	remaining: 16.7s
494:	learn: 0.1719039	total: 16.4s	remaining: 16.7s
495:	learn: 0.1718449	total: 16.4s	remaining: 16.7s
496:	learn: 0.1716808	total: 16.4s	remaining: 16.6s
497:	learn: 0.1715857	total: 16.5s	remaining: 16.6s
498:	learn: 0.1714201	total: 16.5s	remaining: 16.6s
499:	learn: 0.1713228	total: 16.5s	remaining: 16.5s
500:	learn: 0.1712635	total: 16.6s	remaining: 16.5s
501:	learn: 0.1712158	total: 16.6s	remaining: 16.5s
502:	learn: 0.1710215	total: 16.6s	remaining: 16.4s
503:	learn: 0.1708293	total: 16.7s	remaining: 16.4s
504:	learn: 0.1707584	total: 16.7s	remaining: 16.4s
505:	learn: 0.

647:	learn: 0.1532939	total: 22.1s	remaining: 12s
648:	learn: 0.1532565	total: 22.1s	remaining: 12s
649:	learn: 0.1530420	total: 22.1s	remaining: 11.9s
650:	learn: 0.1529074	total: 22.2s	remaining: 11.9s
651:	learn: 0.1527653	total: 22.2s	remaining: 11.9s
652:	learn: 0.1526328	total: 22.2s	remaining: 11.8s
653:	learn: 0.1525268	total: 22.3s	remaining: 11.8s
654:	learn: 0.1525060	total: 22.3s	remaining: 11.8s
655:	learn: 0.1523427	total: 22.4s	remaining: 11.7s
656:	learn: 0.1522224	total: 22.4s	remaining: 11.7s
657:	learn: 0.1521816	total: 22.4s	remaining: 11.7s
658:	learn: 0.1520616	total: 22.4s	remaining: 11.6s
659:	learn: 0.1519910	total: 22.5s	remaining: 11.6s
660:	learn: 0.1519148	total: 22.5s	remaining: 11.6s
661:	learn: 0.1517961	total: 22.6s	remaining: 11.5s
662:	learn: 0.1517391	total: 22.6s	remaining: 11.5s
663:	learn: 0.1514482	total: 22.6s	remaining: 11.4s
664:	learn: 0.1513434	total: 22.7s	remaining: 11.4s
665:	learn: 0.1513093	total: 22.7s	remaining: 11.4s
666:	learn: 0.15

...

199:	learn: 0.2094266	total: 8.5s	remaining: 34s
200:	learn: 0.2091727	total: 8.55s	remaining: 34s
201:	learn: 0.2089245	total: 8.6s	remaining: 34s
202:	learn: 0.2085478	total: 8.65s	remaining: 34s
203:	learn: 0.2082571	total: 8.71s	remaining: 34s
204:	learn: 0.2080355	total: 8.76s	remaining: 34s
205:	learn: 0.2078371	total: 8.81s	remaining: 34s
206:	learn: 0.2074488	total: 8.89s	remaining: 34s
207:	learn: 0.2072135	total: 8.97s	remaining: 34.1s
208:	learn: 0.2068940	total: 9.02s	remaining: 34.1s
209:	learn: 0.2068563	total: 9.07s	remaining: 34.1s
210:	learn: 0.2067256	total: 9.11s	remaining: 34.1s
211:	learn: 0.2065924	total: 9.16s	remaining: 34s
212:	learn: 0.2064045	total: 9.21s	remaining: 34s
213:	learn: 0.2062979	total: 9.25s	remaining: 34s
214:	learn: 0.2062043	total: 9.3s	remaining: 34s
215:	learn: 0.2060359	total: 9.35s	remaining: 33.9s
216:	learn: 0.2058435	total: 9.4s	remaining: 33.9s
217:	learn: 0.2056586	total: 9.45s	remaining: 33.9s
218:	learn: 0.2055975	total: 9.49s	remai

359:	learn: 0.1815354	total: 15.6s	remaining: 27.7s
360:	learn: 0.1812805	total: 15.6s	remaining: 27.7s
361:	learn: 0.1811023	total: 15.7s	remaining: 27.6s
362:	learn: 0.1810138	total: 15.7s	remaining: 27.6s
363:	learn: 0.1809228	total: 15.8s	remaining: 27.5s
364:	learn: 0.1807259	total: 15.8s	remaining: 27.5s
365:	learn: 0.1805010	total: 15.9s	remaining: 27.5s
366:	learn: 0.1803715	total: 15.9s	remaining: 27.5s
367:	learn: 0.1802555	total: 16s	remaining: 27.4s
368:	learn: 0.1801913	total: 16s	remaining: 27.4s
369:	learn: 0.1799888	total: 16s	remaining: 27.3s
370:	learn: 0.1799218	total: 16.1s	remaining: 27.3s
371:	learn: 0.1797481	total: 16.1s	remaining: 27.2s
372:	learn: 0.1797040	total: 16.1s	remaining: 27.1s
373:	learn: 0.1795057	total: 16.2s	remaining: 27.1s
374:	learn: 0.1793294	total: 16.2s	remaining: 27s
375:	learn: 0.1792747	total: 16.3s	remaining: 27s
376:	learn: 0.1791836	total: 16.3s	remaining: 26.9s
377:	learn: 0.1790516	total: 16.3s	remaining: 26.9s
378:	learn: 0.1789463	

520:	learn: 0.1618133	total: 21.5s	remaining: 19.8s
521:	learn: 0.1617552	total: 21.5s	remaining: 19.7s
522:	learn: 0.1616601	total: 21.6s	remaining: 19.7s
523:	learn: 0.1616313	total: 21.6s	remaining: 19.6s
524:	learn: 0.1613645	total: 21.7s	remaining: 19.6s
525:	learn: 0.1613548	total: 21.7s	remaining: 19.5s
526:	learn: 0.1611707	total: 21.7s	remaining: 19.5s
527:	learn: 0.1610143	total: 21.8s	remaining: 19.5s
528:	learn: 0.1608967	total: 21.8s	remaining: 19.4s
529:	learn: 0.1607919	total: 21.9s	remaining: 19.4s
530:	learn: 0.1605988	total: 21.9s	remaining: 19.3s
531:	learn: 0.1604011	total: 21.9s	remaining: 19.3s
532:	learn: 0.1603120	total: 22s	remaining: 19.3s
533:	learn: 0.1602294	total: 22s	remaining: 19.2s
534:	learn: 0.1600405	total: 22s	remaining: 19.2s
535:	learn: 0.1598861	total: 22.1s	remaining: 19.1s
536:	learn: 0.1597284	total: 22.1s	remaining: 19.1s
537:	learn: 0.1595434	total: 22.2s	remaining: 19s
538:	learn: 0.1592783	total: 22.2s	remaining: 19s
539:	learn: 0.1590948	

682:	learn: 0.1423878	total: 28.4s	remaining: 13.2s
683:	learn: 0.1423088	total: 28.4s	remaining: 13.1s
684:	learn: 0.1420819	total: 28.4s	remaining: 13.1s
685:	learn: 0.1420591	total: 28.5s	remaining: 13s
686:	learn: 0.1418469	total: 28.5s	remaining: 13s
687:	learn: 0.1417681	total: 28.6s	remaining: 12.9s
688:	learn: 0.1416087	total: 28.6s	remaining: 12.9s
689:	learn: 0.1415202	total: 28.6s	remaining: 12.9s
690:	learn: 0.1413427	total: 28.7s	remaining: 12.8s
691:	learn: 0.1412147	total: 28.7s	remaining: 12.8s
692:	learn: 0.1412065	total: 28.8s	remaining: 12.7s
693:	learn: 0.1411298	total: 28.9s	remaining: 12.7s
694:	learn: 0.1410558	total: 29s	remaining: 12.7s
695:	learn: 0.1409579	total: 29.1s	remaining: 12.7s
696:	learn: 0.1409049	total: 29.2s	remaining: 12.7s
697:	learn: 0.1408664	total: 29.2s	remaining: 12.6s
698:	learn: 0.1408221	total: 29.2s	remaining: 12.6s
699:	learn: 0.1407480	total: 29.3s	remaining: 12.6s
700:	learn: 0.1405882	total: 29.3s	remaining: 12.5s
701:	learn: 0.1404

845:	learn: 0.1274702	total: 36.1s	remaining: 6.58s
846:	learn: 0.1273591	total: 36.2s	remaining: 6.54s
847:	learn: 0.1273025	total: 36.2s	remaining: 6.5s
848:	learn: 0.1271930	total: 36.3s	remaining: 6.45s
849:	learn: 0.1270638	total: 36.3s	remaining: 6.41s
850:	learn: 0.1270605	total: 36.4s	remaining: 6.37s
851:	learn: 0.1270049	total: 36.4s	remaining: 6.33s
852:	learn: 0.1268035	total: 36.5s	remaining: 6.28s
853:	learn: 0.1267581	total: 36.5s	remaining: 6.24s
854:	learn: 0.1266047	total: 36.6s	remaining: 6.2s
855:	learn: 0.1265703	total: 36.6s	remaining: 6.16s
856:	learn: 0.1264739	total: 36.7s	remaining: 6.12s
857:	learn: 0.1262421	total: 36.7s	remaining: 6.08s
858:	learn: 0.1261832	total: 36.8s	remaining: 6.04s
859:	learn: 0.1260542	total: 36.8s	remaining: 5.99s
860:	learn: 0.1259658	total: 36.9s	remaining: 5.95s
861:	learn: 0.1259641	total: 36.9s	remaining: 5.91s
862:	learn: 0.1259311	total: 37s	remaining: 5.87s
863:	learn: 0.1259131	total: 37s	remaining: 5.82s
864:	learn: 0.1258

In [64]:
np.mean(cv_scores)

0.9028648621209946

In [65]:
%%time
ctb.fit(train_df_part, y_train_part, cat_features=cat_features_idx);

Learning rate set to 0.016847
0:	learn: 0.6738676	total: 25.8ms	remaining: 25.8s
1:	learn: 0.6568722	total: 51.4ms	remaining: 25.6s
2:	learn: 0.6386390	total: 78.5ms	remaining: 26.1s
3:	learn: 0.6226240	total: 102ms	remaining: 25.5s
4:	learn: 0.6082315	total: 131ms	remaining: 26.1s
5:	learn: 0.5930647	total: 157ms	remaining: 26s
6:	learn: 0.5796675	total: 184ms	remaining: 26.1s
7:	learn: 0.5644235	total: 217ms	remaining: 26.9s
8:	learn: 0.5519232	total: 250ms	remaining: 27.6s
9:	learn: 0.5380081	total: 280ms	remaining: 27.7s
10:	learn: 0.5274532	total: 307ms	remaining: 27.6s
11:	learn: 0.5155046	total: 335ms	remaining: 27.6s
12:	learn: 0.5039250	total: 362ms	remaining: 27.5s
13:	learn: 0.4932726	total: 389ms	remaining: 27.4s
14:	learn: 0.4840909	total: 414ms	remaining: 27.2s
15:	learn: 0.4746328	total: 449ms	remaining: 27.6s
16:	learn: 0.4679718	total: 481ms	remaining: 27.8s
17:	learn: 0.4605335	total: 512ms	remaining: 27.9s
18:	learn: 0.4506446	total: 540ms	remaining: 27.9s
19:	learn:

163:	learn: 0.2193404	total: 4.9s	remaining: 25s
164:	learn: 0.2187669	total: 4.93s	remaining: 25s
165:	learn: 0.2182462	total: 4.97s	remaining: 25s
166:	learn: 0.2179618	total: 5s	remaining: 24.9s
167:	learn: 0.2176099	total: 5.04s	remaining: 24.9s
168:	learn: 0.2175669	total: 5.05s	remaining: 24.8s
169:	learn: 0.2173729	total: 5.08s	remaining: 24.8s
170:	learn: 0.2169070	total: 5.12s	remaining: 24.8s
171:	learn: 0.2166386	total: 5.16s	remaining: 24.8s
172:	learn: 0.2162282	total: 5.19s	remaining: 24.8s
173:	learn: 0.2159926	total: 5.22s	remaining: 24.8s
174:	learn: 0.2155858	total: 5.26s	remaining: 24.8s
175:	learn: 0.2152918	total: 5.29s	remaining: 24.8s
176:	learn: 0.2150468	total: 5.32s	remaining: 24.7s
177:	learn: 0.2147880	total: 5.36s	remaining: 24.8s
178:	learn: 0.2143149	total: 5.39s	remaining: 24.7s
179:	learn: 0.2138547	total: 5.43s	remaining: 24.7s
180:	learn: 0.2136760	total: 5.46s	remaining: 24.7s
181:	learn: 0.2133452	total: 5.49s	remaining: 24.7s
182:	learn: 0.2130991	

325:	learn: 0.1860803	total: 10.9s	remaining: 22.6s
326:	learn: 0.1859826	total: 11s	remaining: 22.6s
327:	learn: 0.1858215	total: 11s	remaining: 22.6s
328:	learn: 0.1855719	total: 11s	remaining: 22.5s
329:	learn: 0.1854465	total: 11.1s	remaining: 22.5s
330:	learn: 0.1853203	total: 11.1s	remaining: 22.5s
331:	learn: 0.1852120	total: 11.1s	remaining: 22.4s
332:	learn: 0.1852009	total: 11.2s	remaining: 22.4s
333:	learn: 0.1850553	total: 11.2s	remaining: 22.4s
334:	learn: 0.1850161	total: 11.3s	remaining: 22.4s
335:	learn: 0.1849254	total: 11.3s	remaining: 22.3s
336:	learn: 0.1849090	total: 11.3s	remaining: 22.3s
337:	learn: 0.1847237	total: 11.4s	remaining: 22.3s
338:	learn: 0.1845678	total: 11.4s	remaining: 22.2s
339:	learn: 0.1843105	total: 11.4s	remaining: 22.2s
340:	learn: 0.1842459	total: 11.5s	remaining: 22.2s
341:	learn: 0.1841436	total: 11.5s	remaining: 22.1s
342:	learn: 0.1840369	total: 11.5s	remaining: 22.1s
343:	learn: 0.1838581	total: 11.6s	remaining: 22.1s
344:	learn: 0.1836

487:	learn: 0.1694416	total: 17.6s	remaining: 18.4s
488:	learn: 0.1693508	total: 17.6s	remaining: 18.4s
489:	learn: 0.1693476	total: 17.6s	remaining: 18.4s
490:	learn: 0.1691632	total: 17.7s	remaining: 18.3s
491:	learn: 0.1690009	total: 17.7s	remaining: 18.3s
492:	learn: 0.1688029	total: 17.8s	remaining: 18.3s
493:	learn: 0.1687524	total: 17.8s	remaining: 18.3s
494:	learn: 0.1687523	total: 17.9s	remaining: 18.2s
495:	learn: 0.1685397	total: 17.9s	remaining: 18.2s
496:	learn: 0.1684339	total: 17.9s	remaining: 18.2s
497:	learn: 0.1683527	total: 18s	remaining: 18.1s
498:	learn: 0.1682278	total: 18s	remaining: 18.1s
499:	learn: 0.1681705	total: 18.1s	remaining: 18.1s
500:	learn: 0.1679081	total: 18.1s	remaining: 18.1s
501:	learn: 0.1676049	total: 18.2s	remaining: 18s
502:	learn: 0.1674459	total: 18.2s	remaining: 18s
503:	learn: 0.1674094	total: 18.3s	remaining: 18s
504:	learn: 0.1673309	total: 18.3s	remaining: 18s
505:	learn: 0.1672840	total: 18.4s	remaining: 17.9s
506:	learn: 0.1671309	to

649:	learn: 0.1523714	total: 23.5s	remaining: 12.7s
650:	learn: 0.1522658	total: 23.5s	remaining: 12.6s
651:	learn: 0.1522207	total: 23.6s	remaining: 12.6s
652:	learn: 0.1522086	total: 23.6s	remaining: 12.5s
653:	learn: 0.1520940	total: 23.6s	remaining: 12.5s
654:	learn: 0.1520356	total: 23.7s	remaining: 12.5s
655:	learn: 0.1519871	total: 23.7s	remaining: 12.4s
656:	learn: 0.1518326	total: 23.7s	remaining: 12.4s
657:	learn: 0.1517804	total: 23.8s	remaining: 12.4s
658:	learn: 0.1517071	total: 23.8s	remaining: 12.3s
659:	learn: 0.1515649	total: 23.8s	remaining: 12.3s
660:	learn: 0.1513758	total: 23.9s	remaining: 12.2s
661:	learn: 0.1510618	total: 23.9s	remaining: 12.2s
662:	learn: 0.1510039	total: 23.9s	remaining: 12.2s
663:	learn: 0.1509924	total: 24s	remaining: 12.1s
664:	learn: 0.1508957	total: 24s	remaining: 12.1s
665:	learn: 0.1508386	total: 24s	remaining: 12.1s
666:	learn: 0.1508003	total: 24.1s	remaining: 12s
667:	learn: 0.1506916	total: 24.1s	remaining: 12s
668:	learn: 0.1505247	

811:	learn: 0.1371717	total: 30s	remaining: 6.93s
812:	learn: 0.1371278	total: 30s	remaining: 6.9s
813:	learn: 0.1370743	total: 30.1s	remaining: 6.87s
814:	learn: 0.1370455	total: 30.1s	remaining: 6.84s
815:	learn: 0.1369748	total: 30.2s	remaining: 6.81s
816:	learn: 0.1369337	total: 30.2s	remaining: 6.78s
817:	learn: 0.1367534	total: 30.3s	remaining: 6.74s
818:	learn: 0.1367172	total: 30.4s	remaining: 6.71s
819:	learn: 0.1366173	total: 30.4s	remaining: 6.68s
820:	learn: 0.1365586	total: 30.5s	remaining: 6.64s
821:	learn: 0.1364208	total: 30.5s	remaining: 6.61s
822:	learn: 0.1363520	total: 30.6s	remaining: 6.58s
823:	learn: 0.1362965	total: 30.7s	remaining: 6.55s
824:	learn: 0.1361247	total: 30.8s	remaining: 6.53s
825:	learn: 0.1360094	total: 30.8s	remaining: 6.5s
826:	learn: 0.1359506	total: 30.9s	remaining: 6.46s
827:	learn: 0.1359171	total: 31s	remaining: 6.43s
828:	learn: 0.1357024	total: 31s	remaining: 6.4s
829:	learn: 0.1356349	total: 31.1s	remaining: 6.36s
830:	learn: 0.1356126	t

974:	learn: 0.1243579	total: 38.4s	remaining: 984ms
975:	learn: 0.1243199	total: 38.4s	remaining: 945ms
976:	learn: 0.1242079	total: 38.5s	remaining: 906ms
977:	learn: 0.1241405	total: 38.5s	remaining: 867ms
978:	learn: 0.1240381	total: 38.6s	remaining: 827ms
979:	learn: 0.1239944	total: 38.6s	remaining: 788ms
980:	learn: 0.1239777	total: 38.6s	remaining: 748ms
981:	learn: 0.1238819	total: 38.7s	remaining: 709ms
982:	learn: 0.1238253	total: 38.7s	remaining: 670ms
983:	learn: 0.1237717	total: 38.8s	remaining: 630ms
984:	learn: 0.1237353	total: 38.8s	remaining: 591ms
985:	learn: 0.1236790	total: 38.9s	remaining: 552ms
986:	learn: 0.1235957	total: 38.9s	remaining: 512ms
987:	learn: 0.1235402	total: 38.9s	remaining: 473ms
988:	learn: 0.1234498	total: 39s	remaining: 433ms
989:	learn: 0.1233979	total: 39s	remaining: 394ms
990:	learn: 0.1233497	total: 39.1s	remaining: 355ms
991:	learn: 0.1232863	total: 39.1s	remaining: 315ms
992:	learn: 0.1232451	total: 39.1s	remaining: 276ms
993:	learn: 0.12

In [67]:
roc_auc_score(y_valid, ctb.predict_proba(valid_df)[:, 1])

0.9190524989858878